In [ ]:
using Gadfly, GaussianProcesses

In [ ]:
srand(201216)
n = 20
X = linspace(-3,3,n)
f = 2*cos(2*X)
Y = [rand(Distributions.Poisson(exp(f[i]))) for i in 1:n]

In [ ]:
#plot the data
plot(x=X,y=Y,Geom.point)

In [ ]:
#GP set-up
k = Matern(3/2,0.0,0.0)   # Matern 3/2 kernel
l = PoisLik()             # Poisson likelihood

In [ ]:
gp = GP(X', vec(Y), MeanZero(), k, l)

In [ ]:
GaussianProcesses.set_priors!(gp.k,[Distributions.Normal(-2.0,4.0),Distributions.Normal(-2.0,4.0)])

In [ ]:
optimize!(gp)

In [ ]:
samples = mcmc(gp;mcrange=Klara.BasicMCRange(nsteps=50000, thinning=10, burnin=10000))

In [ ]:
#Plot posterior samples
xtest = linspace(minimum(gp.X),maximum(gp.X),50);
ymean = [];
fsamples = [];
for i in 1:size(samples,2)
    GaussianProcesses.set_params!(gp,samples[:,i])
    GaussianProcesses.update_target!(gp)
    push!(ymean, predict_y(gp,xtest)[1])
    push!(fsamples,rand(gp,xtest))
end

In [ ]:
#Predictive plots
layers = []
for ym in ymean
    push!(layers, layer(x=xtest,y=ym,Geom.line))
end

plot(layers...,Guide.xlabel("X"),Guide.ylabel("y"))


plot(layer(x=xtest,y=mean(ymean),Geom.line),
     layer(x=X,y=Y,Geom.point))
